# AI: Project 2 - Minesweeper

Leena Dighole

In [1]:
#Import libraries
import numpy as np
import pandas as pd
import itertools
import random
import matplotlib.pylab as plt
from statistics import mean

In [2]:
class mine_environment(object):
    """
    Mine Sweeper board representation
    1. Create Board with given dimension and mine density values
    2. Print text-Based Representation
    3. Check whether cell is safe/ mine
    4. calculate Clue value of the safe cell
    5. calculate no of hidden squares
    
    """
    
    def __init__(self,d,n):   #initialization of the values
        self.dim = int(d)                 # board dimension
        self.mine_density = n             # Mine density
        self.mines = set()                # Set of indexes of the cells containing mines
        self.mines_found = set()          # Mines found by the agent
        self.total_no_of_mines = 0        # Total no of mines on the board
        self.clue = set()                 #Clue value= no of neighboring mine cells
        self.hidden_mines = set()         # Clue value- mines found =hidden mines for particular cell
        #self.grid_array = []           #clue-mine combined array
    
    def create_board(self,d,n):    #create minesweeper board
       
        """
        Randomly creating mines with n density in d*d matrix of the board
        mine_cell_value = 0
        safe_cell_value = 1
        Note: for easy calculation of the clue  its represented in this way.
        
        """
        self.d = int(d)
        self.n = float(n)
        self.board = np.random.choice([0,1],(self.d,self.d), p=[self.n,1-self.n])
        #print(self.board)
        return self.board
    
    def print_original_board(self):    #Print the board
        """
        Print the minesweeper board with text-based representation
        """
        print("The text-based representation of the minesweeper board is:")
        for i in range(self.dim):
            print("--" * self.dim + "-")
            for j in range(self.dim):
                if self.board[i][j] == 0:    #if a mine is located
                    print("|@", end="")
                    self.mines.add((i,j))
                else:
                    print("| ", end="")    #Safe cell
            print("|")
        print("--" * self.dim + "-")
        
        
        #Display the cells with mines
        print('The index of the cells with mines are: \n',self.mines)   # Mine cell index
        self.total_no_of_mines = len(self.mines)
        print('The total no of mines in the board:',self.total_no_of_mines)  #Total no of mines
        
    def print_final_board(self):    #Print the board
        """
        Print the minesweeper board with text-based representation
        """
        self.grid_array = self.board.copy()
        #print("The text-based representation of the complete minesweeper board is:")
        for i in range(self.dim):
            #print("--" * self.dim + "-")
            for j in range(self.dim):
                if self.board[i][j] == 0:    #if a mine is located
                    self.grid_array[i][j] = 100
                    #print("|@", end="")
                    self.mines.add((i,j))
                elif self.board[i][j] == 1:    #if a safe cell
                    clue = self.find_nearby_mines((i,j))
                    x = int(clue)
                    self.grid_array[i][j]=x
                    #print("|",x, end="")
                    clue = None
                    x=None
            #print("|")
        #print("--" * self.dim + "-")
        
        
        #Display the cells with mines
        #print('The index of the cells with mines are: \n',self.mines)   # Mine cell index
        self.total_no_of_mines = len(self.mines)
        #print('The total no of mines in the board:',self.total_no_of_mines)  #Total no of mines
       # print('The grid array will be:\n',self.grid_array)
        
        
    def is_mine(self, cell):   #Check whether given cell is a mine
        """
        Checks if the cell is a safe cell or mine
        """
        i,j = cell
        if self.board[i][j] ==0:
            #print('The cell is mine\n',cell)
            mine = 1
        else:
            mine = 0
        return mine
        
    def find_nearby_mines(self,cell):
        """
        Finding the number of mines present in the adjacent neighbors of the given cell.
        We will consider one column and row adjacent to cell and excluding the given cell, we will search all its surrounding
        8 neighboring cells for the clue value.
        
        """
        #Nearby mine counting variable
        count = 0
        for i in range(cell[0]-1, cell[0]+2):   # neighboring rows
            for j in range(cell[1]-1, cell[1]+2):   #  neighboring columns
                #Exclude the given cell
                if (i,j) ==cell:
                    continue
                
                #Check the boundary of row and columns
                if 0<=i<self.dim  and 0<=j<self.dim:
                    if self.board[i][j] ==0:   #if the neighbor cell is a mine
                        count += 1
        #print('for cell:',cell,'\n The clue value is: ', count)
        
        # The clue value = no of adjacent mines
        self.clue.add((cell,count))
        return count
    
    def agent_won(self):
        """
        Checks if all mines have been correctly identified.
        Agent will be declared as a winner once he successfully able to find all the mines on the board
        """
        if self.mines_found == self.mines:
            print('Hurray!!\n Agent successfully flagged out all the mines :)')
        return self.mines_found == self.mines  
    
    def board_representation(self,d,n):
        """
        Representation of the mine sweeper board
        create and print the board with given dimensions

        """    
        self.create_board(d,n)
        #self.print_original_board() 
        self.print_final_board()
        return self.board,self.mines,self.grid_array
                
                
            
        
 
        
        


In [3]:
class Simple_Agent(object):
    """
    The Improved Agent performs following Operations:
    1. Select any start cell randomly
    2. If the cell is mine, It will add it in no of mines found and keep searching other cells
    3. If cell is safe, It will show its clue value (No of mines in its neighborhood)
    4. If the clue value = total no of neighbor cells => All neighbors are mines, Add it into KB
    5. If the clue value = 0 => All neighbors are safe, Add it into KB
    6. If the clue value < total no of neighbor cells => Perform some operations on the available KB
    7. Collect new inferences with some operations and add it into KB
    8. If no new inference is obtained by new operations, Select new cell to be surfed randomly
    """
    def __init__(self,d,n):
        self.dim = int(d)                #Dimension of board
        self.mine_density = float(n)     #
        self.moves_made = set()          #Track of all the cells visited
        #self.mines= set()        # Track of all mine cells
        self.mines_found = set()   #Track of explored mines
        self.safes = set()       # Track of all safe cells
        self.knowledge = []    #List of equations in the KB which are True
        self.start = []      #start_node of the game
        self.count_total_moves = 0    #Count of moves made
        #self.current_neighbors =set()    #The set of current neighbors of the cell
        self.obj2 = mine_environment(d,n)    #initialization of class mine_environment
        #self.obj3 = Inferences(cell,clue)    #initialization of class Inferences  
        self.output = set()     #final output set of given input d,n
        self.total_set = set()   #set of all index of board
        self.choose_mine_cell = set()   #The clue value of the cell is mine
        self.new_equation = set()   #Improved agent new equations
        self.cell_value = []
        self.cell_list=[]
        self.eqn_list =[]
        self.moves_analysed = set()
        self.equation_set = set()
        self.new_equation_set =set()
        
    def print_move_on_board(self,cell,flag):
        """
        print the current state of the game and revealed cells and theor corresponding clue values on the board

        """    
        print("The text-based representation of the current state of minesweeper board is:")
        i_vector =[]
        j_vector = []
        k =[] 
        count =0
        if flag ==1:   
            sequence = list((cell))
            #print(sequence)
            
            for i,j in sequence:
                i_vector.append(i)
                j_vector.append(j)
            #print(i_vector,j_vector)

                    
            #print the pclue value of all nodes explored till the current state
            for row in range(self.dim):
                print("--" * self.dim + "-")
                for col in range(self.dim):
                    for index in range(len(i_vector)):
                        
                        i = i_vector[index]
                        j = j_vector[index]
                      #  print(i,j)
                        k.append(i)
                        if row == i and col == j:
                            count = 1
                            if self.board[i][j] == 0:    #if a mine is located
                                print("|@", end="")
                                
                            elif self.board[i][j] == 1:    #if a safe cell
                                clue = self.obj2.find_nearby_mines((i,j))
                                #x = int(clue) 
                                print("|",clue, end="")
                                clue =None 
                                x=None
                               
                    if row ==i and col ==self.dim-1:
                        print(end="")
                    else:
                        print("| ", end="")
                

                print("|")
                
            print("--" * self.dim + "-")
        
                    
        elif flag==0:
            i,j = cell
                
                #print the current state
            for row in range(self.dim):
                print("--" * self.dim + "-")
                for col in range(self.dim):
                    if row == i and col == j:
                        if self.board[i][j] == 0:    #if a mine is located
                            print("|@", end="")
                        elif self.board[i][j] == 1:    #if a safe cell
                            clue = self.obj2.find_nearby_mines(cell)
                            x = int(clue)
                            print("|",x, end="")
                            clue = None
                            x=None
                    else:
                        print("| ", end="")    
                print("|")
            print("--" * self.dim + "-")
        
   
        
    
    def mark_mine(self,cell):
        """
        Check whether the given cell is a mine or a safe cell

        """ 
        #Check whether the node is safe or mine
        mine = self.obj2.is_mine(cell)
        if mine:
            self.mines_found.add(cell)
            #print('The mine is located at:', cell)
        else:
            self.safes.add(cell)
            #print('The cell is safe:', cell)
            
        #Add the explored cell into KB
        self.moves_made.add(cell)
        
        return mine
    
    def find_neighbors(self,cell):
        """
        Find all the neighbors of the given cell

        """ 
         #Nearby mine counting variable
        self.current_neighbors =set()    #The set of current neighbors of the cell
        for i in range(cell[0]-1, cell[0]+2):   # neighboring rows
            for j in range(cell[1]-1, cell[1]+2):   #  neighboring columns
                if 0<=i<self.dim  and 0<=j<self.dim:
                    #Exclude the given cell
                    if (i,j) ==cell:
                        continue
                    else:
                        self.current_neighbors.add((i,j))
       # print('The neighbors of cell:',cell,'are: ',self.current_neighbors)
        return self.current_neighbors
    
    def Basic_Inferences(self,cell,clue):
     
        """
        Simple Agent Inferences:
        1.Check whether clue value of cell == no of neighbors => all neighbors are mines
        2.Check whether clue value of cell == 0 => all neighbors are safe
        3. Check whether clue value of cell < no of neighbors => Perform operations and add new inferences in KB
        It has 2 imp parts:
        a) Inferences
        b) Decisions


        """ 
        self.current_neighbors = self.find_neighbors(cell)
        self.neighbor_count = len(list(self.current_neighbors))
        success = None
        game_over = None
        val = None
        
        #If the randomly chosen cell is a mine
        i,j = cell
        #clue_val = self.grid_array[i][j]
        
        if clue =='@':
            clue = 100
            val = 1
        else:
            clue=clue
            val = 0
        
        #Update KB
        self.cell_list.append(cell)
        self.cell_value.append(val)
        #self.eqn_list.append(self.current_neighbors)
        self.moves_analysed.add((cell,val))
        self.equation_set.add((tuple(self.current_neighbors),clue))
       # print('self.equation_set', self.equation_set)
        
        #Retrieve cell and its values
        
        for lhs, rhs in self.equation_set:
            eqn = list(lhs)
            eqn_val = rhs
            # Check the KB and try to simplify the eqns using new updates
            for i in eqn:
                #If the value of a neighbor cell is already solved 
              
                if (i in set(self.cell_list)):
                    k = self.cell_list.index(i)
                    cell_val = self.cell_value[k]
                    #print('subset',i,k,cell_val)   
                    if cell_val ==0:   #cell is safe
                        eqn.remove(i)
                        
                    elif cell_val ==1:   #cell is already determined as a mine
                        eqn.remove(i)
                        eqn_val -= 1
                       
                    #print('new_eqn',eqn)
                            
            if int(eqn_val)== len(eqn):  #if all cells are mine
               # print('all are mines')
                self.mines_found.update(eqn)
                for j in eqn:
                    self.moves_analysed.add((j,1))
                    self.cell_list.append(j)
                    self.cell_value.append(1)
                    #print('cell_list',set(self.cell_list))

            elif int(eqn_val)== 0:  #if all cells are safe
                #print('all are safes')
                self.safes.update( eqn)
                for j in eqn:
                    self.moves_analysed.add((j,0))
                    self.cell_list.append(j)
                    self.cell_value.append(0)

            else:               #when nothing can be inferred
                self.new_equation_set.add((tuple(eqn),eqn_val))

                
        self.equation_set.update(self.new_equation_set)          
        #If all neighbors are mines
        if int(clue) == self.neighbor_count:
            self.mines_found.update( self.current_neighbors)
            #print the new information on the board
            #self.print_move_on_board(self.current_neighbors,1)
            success = 3
            game_over =0
            
        #If all neighbors are safe
        elif int(clue) == 0:
            self.safes.update( self.current_neighbors)
            #self.print_move_on_board(self.current_neighbors,1)
            success = 2
            game_over =0
            
        #If cell is mine   
        elif int(clue)== 100:
            #print('The Simple Agent can not infer the next move for cell:',cell,'\n Lets select the next move randomly')
            success = 1
            game_over =1
        
    
        #If clue is not equal to no of neighbors    
        else:
            #print('The Simple Agent can not infer the next move for cell:',cell,'\n Lets select the next move randomly')
            success = 1
            game_over =0
        
       # print('The Discovered mines are:\n',self.mines_found,'\nThe no of mine cells discovered :', len(self.mines_found))
        #print('The Discovered safe cells are:\n',self.safes,'\nThe no of safe cells discovered :', len(self.safes))
        
        return success,game_over, self.mines_found
        
        
        
   
    def make_random_move(self,cell):
        """
        Make a random move as we have no aditional inference from the given cell
        1. Find all the neighbors of the given cell
        2. Randomly choose a neighbor
        3. Check whether the new cell is safe/mine
        4. If the cell is safe, collect its clue value
        5. print the new state of the board

        """ 
        
        self.current_neighbors = self.find_neighbors(cell)
        new_cell = random.choice(tuple(self.current_neighbors))   #randomly choose the neighboring cells for next move

            #Randomly choosen cell is previousely visited
        if (set(new_cell).issubset(self.cell_list)):    #if all the neighbors are not already traversed
            new_cell = random.choice(tuple(self.current_neighbors.difference(set(new_cell)) ))   #randomly choose the neighboring cells for next move

                
        if (self.cell_list != self.total_set):
            if (self.current_neighbors.issubset(self.cell_list)):    #if all the neighbors are  already traversed
                new_cell = random.choice(tuple(self.total_set.difference(self.cell_list) ))   #choose random cell from not explored cell set



            #if the new randomly choosen cell is a safe cell, collect its clue value
        if not self.mark_mine(new_cell):
            new_clue = self.obj2.find_nearby_mines(new_cell)
        else:
            new_clue = '@'
            self.choose_mine_cell.add(cell)

        #print('The new_cell:',new_cell,'\n The new_clue: ',new_clue)

            #print the current state of the board
        self.current_state = self.moves_made
      
        return new_cell, new_clue
        
       
    
    def add_knowledge(self,cell):
        """
        Start the game for Simple Agent
        1. start at random cell
        2. Keep exploring the safe nodes if we have any new inferences
        3. Else choose a random cell and explore further
        
        Inferences according to simple agents Knowledge:
        1. success =2 :all the neighboring cells of a cell are safe 
            so choose any random neighbor cell for next step
        2. success =3 :all the neighboring cells of a cell are mines 
            so choose any random  cell  from the board for next step
        3. success =1 :Agent is unable to decide whether all neighbors are safe/ mines 
            so choose any random neighbor cell for next step
        4. success =0 :Agent is able to find all mines on the board
            Terminate the game (Agent won the game!!)
        
        """ 
        self.count_total_moves = 0
        #Start game from random cell
        cell= self.start
        #self.print_move_on_board(tuple(cell),0)
        new_cell, new_clue = self.make_random_move(cell)   #Make a random move from first cell
        success, game_over ,self.mines_found =self.Basic_Inferences(new_cell,new_clue)
        cell= new_cell
        
        #Untill all mines are found
        
        while set(self.cell_list) != self.total_set:
            
            #print('The mines found are: ',self.mines_found)
            
            while success and set(self.cell_list) != self.total_set:
                self.count_total_moves +=1
                
            
                if success== 2:   # All neighboring cells are safe
                    
                    new_cell, new_clue = self.make_random_move(cell)
                    success, game_over ,self.mines_found =self.Basic_Inferences(new_cell,new_clue)
                    cell = new_cell
                    continue
                
                if success== 1:   # If agent is unable to decide then it should randomly pick the neighboring cell
                    new_cell, new_clue = self.make_random_move(cell)
                    success, game_over ,self.mines_found =self.Basic_Inferences(new_cell,new_clue)
                    cell=new_cell
                    continue
                    
                #If all neighboring cells are mine, choose the random cell from board
                if success == 3:
                    i = np.random.choice(self.dim)
                    j = np.random.choice(self.dim)
                    cell = (i,j)
                    new_cell, new_clue = self.make_random_move(cell)
                    success, game_over ,self.mines_found =self.Basic_Inferences(new_cell,new_clue)
                    cell = new_cell
                    continue
                
               
                    
                if self.mines_found == self.total_mines:   #agent successfully found all mines
                    success = 0
                    #flag = 0
                    #print('The Agent has successfully found all mines in steps:',self.count_total_moves)
                    exit()
        if success ==0:
            #flag =0
            exit()
             
    
                    
        return self.count_total_moves
        
    def add_knowledge_bonus(self,cell):
        """
        Start the game for Simple Agent
        1. start at random cell
        2. Keep exploring the safe nodes if we have any new inferences
        3. Else choose a random cell and explore further
        4. Make moves based upon knowledge of no of total mines
        
        Inferences according to simple agents Knowledge:
        1. success =2 :all the neighboring cells of a cell are safe 
            so choose any random neighbor cell for next step
        2. success =3 :all the neighboring cells of a cell are mines 
            so choose any random  cell  from the board for next step
        3. success =1 :Agent is unable to decide whether all neighbors are safe/ mines 
            so choose any random neighbor cell for next step
        4. success =0 :Agent is able to find all mines on the board
            Terminate the game (Agent won the game!!)
        
        """ 
        self.count_total_moves = 0
        #Start game from random cell
        cell= self.start
        #self.print_move_on_board(tuple(cell),0)
        new_cell, new_clue = self.make_random_move(cell)   #Make a random move from first cell
        success, game_over ,self.mines_found =self.Basic_Inferences(new_cell,new_clue)
        cell= new_cell
        
        #Untill all mines are found

        while self.mines_found != self.total_mines:
          
            while success and self.mines_found != self.total_mines:
                self.count_total_moves +=1
                
            
                if success== 2:   # All neighboring cells are safe
                    
                    new_cell, new_clue = self.make_random_move(cell)
                    success, game_over ,self.mines_found =self.Basic_Inferences(new_cell,new_clue)
                    cell = new_cell
                    continue
                
                if success== 1:   # If agent is unable to decide then it should randomly pick the neighboring cell
                    new_cell, new_clue = self.make_random_move(cell)
                    success, game_over ,self.mines_found =self.Basic_Inferences(new_cell,new_clue)
                    cell=new_cell
                    continue
                    
                #If all neighboring cells are mine, choose the random cell from board
                if success == 3:
                    i = np.random.choice(self.dim)
                    j = np.random.choice(self.dim)
                    cell = (i,j)
                    new_cell, new_clue = self.make_random_move(cell)
                    success, game_over ,self.mines_found =self.Basic_Inferences(new_cell,new_clue)
                    cell = new_cell
                    continue
                    
                if self.mines_found == self.total_mines:   #agent successfully found all mines
                    success = 0
                    #flag = 0
                   # print('The Agent has successfully found all mines in steps:',self.count_total_moves)
                    exit()
        if success ==0:
            #flag =0
            exit()
             
    
                    
        return self.count_total_moves
              
    
    def end_game(self):
        """
        Agent has successfully identified all the mines of the game board.
        Lets end the game by collecting d,n, and mines found and use them for calculations

        """ 
        
        print('Hurray!!\n Agent successfully flagged out all the mines :)\n')
        print('for d:',d,'and for mine density:',n,'\n the total no of mines:',len(self.mines_found),'\n The Simple Agent is successfully able to find the mines in steps:',self.count_total_moves)

        self.output.add((d,n,len(self.mines_found),self.count_total_moves))
        self.choose_mine_cell.clear()   
        return d,n,len(self.mines_found),self.count_total_moves, self.output
    
    def calculations(self,d,n):
        """
        Once the agent has discovered total mines on the board, lets calculate:
        1. Final score = total no of mines discovered/total mines
        2. Safe cells score = total no  of safe cells identified/total safe cells

        """
        
        #Calculation of safely identified mines/total mines
        #self.final_score =None
        self.final_score_1 =None
      
        #method-2
       # print('self.moves_made\n:',self.moves_made,'length:',len(self.moves_made))
       # print('self.mines_found\n:',self.mines_found,'length:',len(self.mines_found))
        
        #print('len of mines..Leena u r on a right track',self.mines_found == self.total_mines)
        self.t = self.mines_found.difference(self.moves_made)
        #print('the safely identified mines are:\n:',self.t, 'Its length is:',len(self.t))
        
        if len(self.total_mines):
            self.final_score_1 = len(self.t)/len(self.total_mines)
            print('The Final score_1 for d:',d,'\n and for mine density n:',n,'\n is:',len(self.t),'/',len(self.total_mines),'=',self.final_score_1)
       
             
        
        return self.final_score_1
          

    def start_game(self,d,n):
        """
        Representation of the mine sweeper board- create and print the board with given dimensions
        Select the start cell of the game randomly
        Check whether its mine or a safe cell
        Complete the search of all cells and calculate total no of mines

        """    
        self.board, self.total_mines, self.grid_array = self.obj2.board_representation(d,n)
        
        #Creating set containing all the index of the board dimension
        for i in range(self.dim):
            for j in range(self.dim):
                self.total_set.add((i,j))
        
        #Randomly creating first start cell
        i = np.random.choice(self.dim)
        j = np.random.choice(self.dim)
        self.start = (i,j)
        print('The start node is:',self.start)
        self.count_total_moves = self.add_knowledge(self.start)
        self.end_game()
        self.score1 = self.calculations(d,n)
        return len(self.moves_made),self.score1,self.count_total_moves
  
    def start_game_bonus(self,d,n):
        """
        Representation of the mine sweeper board- create and print the board with given dimensions
        Select the start cell of the game randomly
        Check whether its mine or a safe cell
        Complete the search of all cells and calculate total no of mines

        """    
        self.board, self.total_mines, self.grid_array = self.obj2.board_representation(d,n)
        
        #Creating set containing all the index of the board dimension
        for i in range(self.dim):
            for j in range(self.dim):
                self.total_set.add((i,j))
        
        #Randomly creating first start cell
        i = np.random.choice(self.dim)
        j = np.random.choice(self.dim)
        self.start = (i,j)
        print('The start node is:',self.start)
        self.count_total_moves = self.add_knowledge_bonus(self.start)
        self.end_game()
        self.score1 = self.calculations(d,n)
        return len(self.moves_made),self.score1,self.count_total_moves
    
        
        
        
        
     

In [ ]:
#Start the minesweeper Game!!
d = input('Enter the size of dimension of the minesweeper d :')   
n=input('Enter the mine density n :')
obj3 = Simple_Agent(d,n)
obj3.start_game_bonus(d,n)

In [ ]:
#Start the minesweeper Game!!
d = input('Enter the size of dimension of the minesweeper d :')   
n=input('Enter the mine density n :')
obj3 = Simple_Agent(d,n)
obj3.start_game(d,n)


In [ ]:
# Performance of Simple Agent
#Start the Game
record_steps = []
n_set = []
score_set_1 =[]
count_steps =[]
d = 20
#n = 0.1
for n in range(9):
    n = (n+1)/10
    
    n_set.append(n)
    obj1 = Simple_Agent(d,n)
    i,j,k=obj1.start_game(d,n)
    record_steps.append(i)
    score_set_1.append(j)
    count_steps.append(k)
    
print('n:', n_set)
print('steps:',record_steps)
print('score_1:',score_set_1)
print('count_steps:',count_steps)

  

In [6]:
# Performance of Simple Agent
#Start the Game
records = []
n_set = []
final_score =[]
count =[]

d = 10
#n = 0.1
for n in range(9):
    n = (n+1)/10
    n_set.append(n)
    score_set_1 =[]
    #count_steps =[]
    record_steps = []
    for r in range(50):
        obj1 = Simple_Agent(d,n)
       
        i,j,k=obj1.start_game(d,n)
        record_steps.append(i)
        score_set_1.append(j)
        #count_steps.append(k)
        
    records.append(mean(record_steps))
    final_score.append(mean(score_set_1))
    #count.append(mean(count_steps))
    
    
print('n:', n_set)
print('steps:',records)
print('score_1:',final_score)
#print('count_steps:',count)


The start node is: (9, 6)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 9 
 The Simple Agent is successfully able to find the mines in steps: 40
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 5 / 9 = 0.5555555555555556
The start node is: (5, 1)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 15 
 The Simple Agent is successfully able to find the mines in steps: 56
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 6 / 15 = 0.4
The start node is: (9, 8)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 10 
 The Simple Agent is successfully able to find the mines in steps: 37
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 5 / 10 = 0.5
The start node is: (2, 4)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 a

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 6 
 The Simple Agent is successfully able to find the mines in steps: 29
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 2 / 6 = 0.3333333333333333
The start node is: (2, 5)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 4 
 The Simple Agent is successfully able to find the mines in steps: 28
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 2 / 4 = 0.5
The start node is: (7, 8)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 12 
 The Simple Agent is successfully able to find the mines in steps: 45
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 3 / 12 = 0.25
The start node is: (2, 8)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 17 
 The Simple Agent is successfully able to find the mines in steps: 53
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 8 / 17 = 0.47058823529411764
The start node is: (4, 0)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 19 
 The Simple Agent is successfully able to find the mines in steps: 48
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 9 / 19 = 0.47368421052631576
The start node is: (4, 2)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 22 
 The Simple Agent is successfully able to find the mines in steps: 68
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 7 / 22 = 0.3181818181818182
The start node is: (3, 2)
Hurray!!
 Agent successfully flagged out all the mines :)

for

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 27 
 The Simple Agent is successfully able to find the mines in steps: 65
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 15 / 27 = 0.5555555555555556
The start node is: (3, 6)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 21 
 The Simple Agent is successfully able to find the mines in steps: 54
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 7 / 21 = 0.3333333333333333
The start node is: (0, 5)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 19 
 The Simple Agent is successfully able to find the mines in steps: 49
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 10 / 19 = 0.5263157894736842
The start node is: (4, 9)
Hurray!!
 Agent successfully flagged out all the mines :)

for

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.3 
 the total no of mines: 34 
 The Simple Agent is successfully able to find the mines in steps: 96
The Final score_1 for d: 10 
 and for mine density n: 0.3 
 is: 16 / 34 = 0.47058823529411764
The start node is: (3, 6)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.3 
 the total no of mines: 39 
 The Simple Agent is successfully able to find the mines in steps: 94
The Final score_1 for d: 10 
 and for mine density n: 0.3 
 is: 11 / 39 = 0.28205128205128205
The start node is: (7, 6)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.3 
 the total no of mines: 25 
 The Simple Agent is successfully able to find the mines in steps: 64
The Final score_1 for d: 10 
 and for mine density n: 0.3 
 is: 7 / 25 = 0.28
The start node is: (7, 1)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and f

...

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.7 
 the total no of mines: 68 
 The Simple Agent is successfully able to find the mines in steps: 105
The Final score_1 for d: 10 
 and for mine density n: 0.7 
 is: 22 / 68 = 0.3235294117647059
The start node is: (5, 0)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.7 
 the total no of mines: 74 
 The Simple Agent is successfully able to find the mines in steps: 90
The Final score_1 for d: 10 
 and for mine density n: 0.7 
 is: 28 / 74 = 0.3783783783783784
The start node is: (6, 3)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.7 
 the total no of mines: 67 
 The Simple Agent is successfully able to find the mines in steps: 98
The Final score_1 for d: 10 
 and for mine density n: 0.7 
 is: 24 / 67 = 0.3582089552238806
The start node is: (4, 7)
Hurray!!
 Agent successfully flagged out all the mines :)

f

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 90 
 The Simple Agent is successfully able to find the mines in steps: 142
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 16 / 90 = 0.17777777777777778
The start node is: (8, 1)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 82 
 The Simple Agent is successfully able to find the mines in steps: 111
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 19 / 82 = 0.23170731707317074
The start node is: (9, 6)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 80 
 The Simple Agent is successfully able to find the mines in steps: 125
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 9 / 80 = 0.1125
The start node is: (5, 9)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 85 
 The Simple Agent is successfully able to find the mines in steps: 135
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 21 / 85 = 0.24705882352941178
The start node is: (3, 3)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 83 
 The Simple Agent is successfully able to find the mines in steps: 113
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 23 / 83 = 0.27710843373493976
The start node is: (4, 9)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 85 
 The Simple Agent is successfully able to find the mines in steps: 126
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 27 / 85 = 0.3176470588235294
The start node is: (4, 5)
Hurray!!
 Agent successfully flagged out all the mines :

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.9 
 the total no of mines: 92 
 The Simple Agent is successfully able to find the mines in steps: 157
The Final score_1 for d: 10 
 and for mine density n: 0.9 
 is: 13 / 92 = 0.14130434782608695
The start node is: (6, 4)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.9 
 the total no of mines: 90 
 The Simple Agent is successfully able to find the mines in steps: 176
The Final score_1 for d: 10 
 and for mine density n: 0.9 
 is: 9 / 90 = 0.1
The start node is: (2, 1)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.9 
 the total no of mines: 88 
 The Simple Agent is successfully able to find the mines in steps: 114
The Final score_1 for d: 10 
 and for mine density n: 0.9 
 is: 24 / 88 = 0.2727272727272727
The start node is: (9, 3)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and 

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.9 
 the total no of mines: 91 
 The Simple Agent is successfully able to find the mines in steps: 139
The Final score_1 for d: 10 
 and for mine density n: 0.9 
 is: 15 / 91 = 0.16483516483516483
The start node is: (2, 4)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.9 
 the total no of mines: 90 
 The Simple Agent is successfully able to find the mines in steps: 117
The Final score_1 for d: 10 
 and for mine density n: 0.9 
 is: 22 / 90 = 0.24444444444444444
The start node is: (5, 2)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.9 
 the total no of mines: 94 
 The Simple Agent is successfully able to find the mines in steps: 159
The Final score_1 for d: 10 
 and for mine density n: 0.9 
 is: 6 / 94 = 0.06382978723404255
The start node is: (6, 7)
Hurray!!
 Agent successfully flagged out all the mines :

In [7]:
#Bonus Question: Knowing no of mines, Performance of Simple Agent
# Performance of Simple Agent
#Start the Game
records = []
n_set = []
final_score =[]
count =[]

d = 10
#n = 0.1
for n in range(9):
    n = (n+1)/10
    n_set.append(n)
    score_set_1 =[]
    #count_steps =[]
    record_steps = []
    for r in range(50):
        obj1 = Simple_Agent(d,n)
       
        i,j,k=obj1.start_game_bonus(d,n)
        record_steps.append(i)
        score_set_1.append(j)
        #count_steps.append(k)
        
    records.append(mean(record_steps))
    final_score.append(mean(score_set_1))
    #count.append(mean(count_steps))
    
    
print('n:', n_set)
print('steps:',records)
print('score_1:',final_score)
#print('count_steps:',count)





The start node is: (7, 9)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 11 
 The Simple Agent is successfully able to find the mines in steps: 42
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 4 / 11 = 0.36363636363636365
The start node is: (0, 3)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 9 
 The Simple Agent is successfully able to find the mines in steps: 25
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 5 / 9 = 0.5555555555555556
The start node is: (1, 0)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 9 
 The Simple Agent is successfully able to find the mines in steps: 32
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 3 / 9 = 0.3333333333333333
The start node is: (9, 3)
Hurray!!
 Agent successfully flagged out 

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 6 
 The Simple Agent is successfully able to find the mines in steps: 34
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 2 / 6 = 0.3333333333333333
The start node is: (8, 0)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 12 
 The Simple Agent is successfully able to find the mines in steps: 39
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 4 / 12 = 0.3333333333333333
The start node is: (6, 0)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 7 
 The Simple Agent is successfully able to find the mines in steps: 33
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 2 / 7 = 0.2857142857142857
The start node is: (6, 3)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 14 
 The Simple Agent is successfully able to find the mines in steps: 49
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 5 / 14 = 0.35714285714285715
The start node is: (7, 3)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 20 
 The Simple Agent is successfully able to find the mines in steps: 53
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 9 / 20 = 0.45
The start node is: (5, 8)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 22 
 The Simple Agent is successfully able to find the mines in steps: 56
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 6 / 22 = 0.2727272727272727
The start node is: (5, 6)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for 

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 17 
 The Simple Agent is successfully able to find the mines in steps: 41
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 7 / 17 = 0.4117647058823529
The start node is: (4, 2)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 15 
 The Simple Agent is successfully able to find the mines in steps: 48
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 3 / 15 = 0.2
The start node is: (3, 6)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 22 
 The Simple Agent is successfully able to find the mines in steps: 57
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 6 / 22 = 0.2727272727272727
The start node is: (7, 7)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mi

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.3 
 the total no of mines: 33 
 The Simple Agent is successfully able to find the mines in steps: 66
The Final score_1 for d: 10 
 and for mine density n: 0.3 
 is: 17 / 33 = 0.5151515151515151
The start node is: (2, 0)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.3 
 the total no of mines: 26 
 The Simple Agent is successfully able to find the mines in steps: 64
The Final score_1 for d: 10 
 and for mine density n: 0.3 
 is: 14 / 26 = 0.5384615384615384
The start node is: (9, 8)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.3 
 the total no of mines: 36 
 The Simple Agent is successfully able to find the mines in steps: 72
The Final score_1 for d: 10 
 and for mine density n: 0.3 
 is: 13 / 36 = 0.3611111111111111
The start node is: (2, 8)
Hurray!!
 Agent successfully flagged out all the mines :)

fo

...

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.7 
 the total no of mines: 70 
 The Simple Agent is successfully able to find the mines in steps: 109
The Final score_1 for d: 10 
 and for mine density n: 0.7 
 is: 16 / 70 = 0.22857142857142856
The start node is: (7, 7)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.7 
 the total no of mines: 67 
 The Simple Agent is successfully able to find the mines in steps: 90
The Final score_1 for d: 10 
 and for mine density n: 0.7 
 is: 29 / 67 = 0.43283582089552236
The start node is: (0, 9)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.7 
 the total no of mines: 68 
 The Simple Agent is successfully able to find the mines in steps: 107
The Final score_1 for d: 10 
 and for mine density n: 0.7 
 is: 20 / 68 = 0.29411764705882354
The start node is: (5, 8)
Hurray!!
 Agent successfully flagged out all the mines :

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 82 
 The Simple Agent is successfully able to find the mines in steps: 155
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 18 / 82 = 0.21951219512195122
The start node is: (2, 6)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 87 
 The Simple Agent is successfully able to find the mines in steps: 152
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 14 / 87 = 0.16091954022988506
The start node is: (8, 8)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 79 
 The Simple Agent is successfully able to find the mines in steps: 110
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 21 / 79 = 0.26582278481012656
The start node is: (0, 1)
Hurray!!
 Agent successfully flagged out all the mines 

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 78 
 The Simple Agent is successfully able to find the mines in steps: 115
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 21 / 78 = 0.2692307692307692
The start node is: (8, 9)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 77 
 The Simple Agent is successfully able to find the mines in steps: 127
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 16 / 77 = 0.2077922077922078
The start node is: (9, 2)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 73 
 The Simple Agent is successfully able to find the mines in steps: 137
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 15 / 73 = 0.2054794520547945
The start node is: (2, 2)
Hurray!!
 Agent successfully flagged out all the mines :)


Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.9 
 the total no of mines: 90 
 The Simple Agent is successfully able to find the mines in steps: 142
The Final score_1 for d: 10 
 and for mine density n: 0.9 
 is: 12 / 90 = 0.13333333333333333
The start node is: (0, 2)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.9 
 the total no of mines: 92 
 The Simple Agent is successfully able to find the mines in steps: 135
The Final score_1 for d: 10 
 and for mine density n: 0.9 
 is: 18 / 92 = 0.1956521739130435
The start node is: (6, 9)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.9 
 the total no of mines: 88 
 The Simple Agent is successfully able to find the mines in steps: 146
The Final score_1 for d: 10 
 and for mine density n: 0.9 
 is: 12 / 88 = 0.13636363636363635
The start node is: (2, 2)
Hurray!!
 Agent successfully flagged out all the mines :

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.9 
 the total no of mines: 89 
 The Simple Agent is successfully able to find the mines in steps: 158
The Final score_1 for d: 10 
 and for mine density n: 0.9 
 is: 7 / 89 = 0.07865168539325842
The start node is: (3, 3)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.9 
 the total no of mines: 90 
 The Simple Agent is successfully able to find the mines in steps: 139
The Final score_1 for d: 10 
 and for mine density n: 0.9 
 is: 16 / 90 = 0.17777777777777778
The start node is: (5, 5)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.9 
 the total no of mines: 86 
 The Simple Agent is successfully able to find the mines in steps: 104
The Final score_1 for d: 10 
 and for mine density n: 0.9 
 is: 29 / 86 = 0.3372093023255814
The start node is: (8, 7)
Hurray!!
 Agent successfully flagged out all the mines :)

In [8]:
class Improved_Agent(object):
    """
    The Improved Agent performs following Operations:
    1. Select any start cell randomly
    2. If the cell is mine, It will add it in no of mines found and keep searching other cells
    3. If cell is safe, It will show its clue value (No of mines in its neighborhood)
    4. If the clue value = total no of neighbor cells => All neighbors are mines, Add it into KB
    5. If the clue value = 0 => All neighbors are safe, Add it into KB
    6. If the clue value < total no of neighbor cells => Perform some operations on the available KB
    7. Collect new inferences with some operations and add it into KB
    8. If no new inference is obtained by new operations, Select new cell to be surfed randomly
    """
    def __init__(self,d,n):
        self.dim = int(d)                #Dimension of board
        self.mine_density = float(n)     #
        self.moves_made = set()          #Track of all the cells visited
        #self.mines= set()        # Track of all mine cells
        self.mines_found = set()   #Track of explored mines
        self.safes = set()       # Track of all safe cells
        self.knowledge = []    #List of equations in the KB which are True
        self.start = []      #start_node of the game
        self.count_total_moves = 0    #Count of moves made
        #self.current_neighbors =set()    #The set of current neighbors of the cell
        self.obj2 = mine_environment(d,n)    #initialization of class mine_environment
        #self.obj3 = Inferences(cell,clue)    #initialization of class Inferences  
        self.output = set()     #final output set of given input d,n
        self.total_set = set()   #set of all index of board
        self.choose_mine_cell = set()   #The clue value of the cell is mine
        self.new_equation = set()   #Improved agent new equations
        self.cell_value = []
        self.cell_list=[]
        self.eqn_list =[]
        self.moves_analysed = set()
        self.equation_set = set()
        self.new_equation_set =set()
        
    def print_move_on_board(self,cell,flag):
        """
        print the current state of the game and revealed cells and theor corresponding clue values on the board

        """    
        print("The text-based representation of the current state of minesweeper board is:")
        i_vector =[]
        j_vector = []
        k =[] 
        count =0
        if flag ==1:   
            sequence = list((cell))
            #print(sequence)
            
            for i,j in sequence:
                i_vector.append(i)
                j_vector.append(j)
            #print(i_vector,j_vector)

                    
            #print the pclue value of all nodes explored till the current state
            for row in range(self.dim):
                print("--" * self.dim + "-")
                for col in range(self.dim):
                    for index in range(len(i_vector)):
                        
                        i = i_vector[index]
                        j = j_vector[index]
                      #  print(i,j)
                        k.append(i)
                        if row == i and col == j:
                            count = 1
                            if self.board[i][j] == 0:    #if a mine is located
                                print("|@", end="")
                                
                            elif self.board[i][j] == 1:    #if a safe cell
                                clue = self.obj2.find_nearby_mines((i,j))
                                #x = int(clue) 
                                print("|",clue, end="")
                                clue =None 
                                x=None
                               
                    if row ==i and col ==self.dim-1:
                        print(end="")
                    else:
                        print("| ", end="")
                

                print("|")
                
            print("--" * self.dim + "-")
        
                    
        elif flag==0:
            i,j = cell
                
                #print the current state
            for row in range(self.dim):
                print("--" * self.dim + "-")
                for col in range(self.dim):
                    if row == i and col == j:
                        if self.board[i][j] == 0:    #if a mine is located
                            print("|@", end="")
                        elif self.board[i][j] == 1:    #if a safe cell
                            clue = self.obj2.find_nearby_mines(cell)
                            x = int(clue)
                            print("|",x, end="")
                            clue = None
                            x=None
                    else:
                        print("| ", end="")    
                print("|")
            print("--" * self.dim + "-")
        
   
        
    
    def mark_mine(self,cell):
        """
        Check whether the given cell is a mine or a safe cell

        """ 
        #Check whether the node is safe or mine
        mine = self.obj2.is_mine(cell)
        if mine:
            self.mines_found.add(cell)
            #print('The mine is located at:', cell)
        else:
            self.safes.add(cell)
           # print('The cell is safe:', cell)
            
        #Add the explored cell into KB
        self.moves_made.add(cell)
        
        return mine
    
    def find_neighbors(self,cell):
        """
        Find all the neighbors of the given cell

        """ 
         #Nearby mine counting variable
        self.current_neighbors =set()    #The set of current neighbors of the cell
        for i in range(cell[0]-1, cell[0]+2):   # neighboring rows
            for j in range(cell[1]-1, cell[1]+2):   #  neighboring columns
                if 0<=i<self.dim  and 0<=j<self.dim:
                    #Exclude the given cell
                    if (i,j) ==cell:
                        continue
                    else:
                        self.current_neighbors.add((i,j))
       # print('The neighbors of cell:',cell,'are: ',self.current_neighbors)
        return self.current_neighbors
    
    def Improved_Inferences(self,cell,clue):
     
        """
        Simple Agent Inferences:
        1.Check whether clue value of cell == no of neighbors => all neighbors are mines
        2.Check whether clue value of cell == 0 => all neighbors are safe
        3. Check whether clue value of cell < no of neighbors => Perform operations and add new inferences in KB
        It has 2 imp parts:
        a) Inferences
        b) Decisions

        """ 
        self.current_neighbors = self.find_neighbors(cell)
        self.neighbor_count = len(list(self.current_neighbors))
        success = None
        game_over = None
        val = None
        
        #If the randomly chosen cell is a mine
        i,j = cell
        #clue_val = self.grid_array[i][j]
        
        if clue =='@':
            clue = 100
            val = 1
        else:
            clue=clue
            val = 0
        
        #Update KB
        self.cell_list.append(cell)
        self.cell_value.append(val)
        #self.eqn_list.append(self.current_neighbors)
        self.moves_analysed.add((cell,val))
        self.equation_set.add((tuple(self.current_neighbors),clue))
       # print('self.equation_set', self.equation_set)
        
        #Retrieve cell and its values
        #Simple agent Basic Inferences
        for lhs, rhs in self.equation_set:
            eqn = list(lhs)
            eqn_val = rhs
            #print('eqn',eqn,eqn_val)
            # Check the KB and try to simplify the eqns using new updates
            for i in eqn:
                #If the value of a neighbor cell is already solved 
                #print('cell_list',set(self.cell_list))
                #print(i)
                #print('if',i in  set(self.cell_list))
                
                if (i in set(self.cell_list)):
                    #print('yeahhh')
                    k = self.cell_list.index(i)
                    cell_val = self.cell_value[k]
                    #print('subset',i,k,cell_val)   
                    if cell_val ==0:   #cell is safe
                        eqn.remove(i)
                        
                    elif cell_val ==1:   #cell is already determined as a mine
                        eqn.remove(i)
                        eqn_val -= 1
                        
                    #print('new_eqn',eqn)
                            
            if int(eqn_val)== len(eqn):  #if all cells are mine
               # print('all are mines')
                self.mines_found.update(eqn)
                for j in eqn:
                    self.moves_analysed.add((j,1))
                    self.cell_list.append(j)
                    self.cell_value.append(1)
                    #print('cell_list',set(self.cell_list))

            elif int(eqn_val)== 0:  #if all cells are safe
                #print('all are safes')
                self.safes.update( eqn)
                for j in eqn:
                    self.moves_analysed.add((j,0))
                    self.cell_list.append(j)
                    self.cell_value.append(0)

            else:               #when nothing can be inferred
                self.new_equation_set.add((tuple(eqn),eqn_val))

                
        self.equation_set.update(self.new_equation_set)
        
        #Improved Agent inferences
        new_val =None
        new_eqn = set()
        for lhs,rhs in self.equation_set:
            
            for l,r in self.equation_set:
                new_eqn = set(lhs).difference_update(set(l))
                
                #if there are some common terms in two equations then try to update new Knowledge from them
                if (new_eqn != None):
                    if (len(set(new_eqn))!= 0): 
                        new_val =0
                
        
        
        
        
        #If all neighbors are mines
        if int(clue) == self.neighbor_count:
            self.mines_found.update( self.current_neighbors)
            #print the new information on the board
            #self.print_move_on_board(self.current_neighbors,1)
            success = 3
            game_over =0
            
        #If all neighbors are safe
        elif int(clue) == 0:
            self.safes.update( self.current_neighbors)
            #self.print_move_on_board(self.current_neighbors,1)
            success = 2
            game_over =0
            
        #If cell is mine   
        elif int(clue)== 100:
            #print('The Simple Agent can not infer the next move for cell:',cell,'\n Lets select the next move randomly')
            success = 1
            game_over =1
        
    
        #If clue is not equal to no of neighbors    
        else:
            #print('The Simple Agent can not infer the next move for cell:',cell,'\n Lets select the next move randomly')
            success = 1
            game_over =0
        
       # print('The Discovered mines are:\n',self.mines_found,'\nThe no of mine cells discovered :', len(self.mines_found))
        #print('The Discovered safe cells are:\n',self.safes,'\nThe no of safe cells discovered :', len(self.safes))
        
        return success,game_over, self.mines_found
        
        
        
   
    def make_random_move(self,cell):
        """
        Make a random move as we have no aditional inference from the given cell
        1. Find all the neighbors of the given cell
        2. Randomly choose a neighbor
        3. Check whether the new cell is safe/mine
        4. If the cell is safe, collect its clue value
        5. print the new state of the board

        """ 
        
        self.current_neighbors = self.find_neighbors(cell)
        new_cell = random.choice(tuple(self.current_neighbors))   #randomly choose the neighboring cells for next move

            #Randomly choosen cell is previousely visited
        if (set(new_cell).issubset(self.cell_list)):    #if all the neighbors are not already traversed
            new_cell = random.choice(tuple(self.current_neighbors.difference(set(new_cell)) ))   #randomly choose the neighboring cells for next move

                
        if (self.cell_list != self.total_set):
            if (self.current_neighbors.issubset(self.cell_list)):    #if all the neighbors are  already traversed
                new_cell = random.choice(tuple(self.total_set.difference(self.cell_list) ))   #choose random cell from not explored cell set



            #if the new randomly choosen cell is a safe cell, collect its clue value
        if not self.mark_mine(new_cell):
            new_clue = self.obj2.find_nearby_mines(new_cell)
        else:
            new_clue = '@'
            self.choose_mine_cell.add(cell)

        #print('The new_cell:',new_cell,'\n The new_clue: ',new_clue)

            #print the current state of the board
        self.current_state = self.moves_made

        return new_cell, new_clue
        
       
    
    def add_knowledge(self,cell):
        """
        Start the game for Simple Agent
        1. start at random cell
        2. Keep exploring the safe nodes if we have any new inferences
        3. Else choose a random cell and explore further
        
        Inferences according to simple agents Knowledge:
        1. success =2 :all the neighboring cells of a cell are safe 
            so choose any random neighbor cell for next step
        2. success =3 :all the neighboring cells of a cell are mines 
            so choose any random  cell  from the board for next step
        3. success =1 :Agent is unable to decide whether all neighbors are safe/ mines 
            so choose any random neighbor cell for next step
        4. success =0 :Agent is able to find all mines on the board
            Terminate the game (Agent won the game!!)
        
        """ 
        self.count_total_moves = 0
        #Start game from random cell
        cell= self.start
        #self.print_move_on_board(tuple(cell),0)
        new_cell, new_clue = self.make_random_move(cell)   #Make a random move from first cell
        success, game_over ,self.mines_found =self.Improved_Inferences(new_cell,new_clue)
        cell= new_cell
        
        #Untill all mines are found
        
        while set(self.cell_list) != self.total_set:
            
            #print('The mines found are: ',self.mines_found)
            
            while success and set(self.cell_list) != self.total_set:
                self.count_total_moves +=1
                
            
                if success== 2:   # All neighboring cells are safe
                    
                    new_cell, new_clue = self.make_random_move(cell)
                    success, game_over ,self.mines_found =self.Improved_Inferences(new_cell,new_clue)
                    cell = new_cell
                    continue
                
                if success== 1:   # If agent is unable to decide then it should randomly pick the neighboring cell
                    new_cell, new_clue = self.make_random_move(cell)
                    success, game_over ,self.mines_found =self.Improved_Inferences(new_cell,new_clue)
                    cell=new_cell
                    continue
                    
                #If all neighboring cells are mine, choose the random cell from board
                if success == 3:
                    i = np.random.choice(self.dim)
                    j = np.random.choice(self.dim)
                    cell = (i,j)
                    new_cell, new_clue = self.make_random_move(cell)
                    success, game_over ,self.mines_found =self.Improved_Inferences(new_cell,new_clue)
                    cell = new_cell
                    continue
                
               
                    
                if self.mines_found == self.total_mines:   #agent successfully found all mines
                    success = 0
                    #flag = 0
                    #print('The Agent has successfully found all mines in steps:',self.count_total_moves)
                    exit()
        if success ==0:
            #flag =0
            exit()
             
    
                    
        return self.count_total_moves
        
    def add_knowledge_bonus(self,cell):
        """
        Start the game for Simple Agent
        1. start at random cell
        2. Keep exploring the safe nodes if we have any new inferences
        3. Else choose a random cell and explore further
        4. Make moves based upon knowledge of no of total mines
        
        Inferences according to simple agents Knowledge:
        1. success =2 :all the neighboring cells of a cell are safe 
            so choose any random neighbor cell for next step
        2. success =3 :all the neighboring cells of a cell are mines 
            so choose any random  cell  from the board for next step
        3. success =1 :Agent is unable to decide whether all neighbors are safe/ mines 
            so choose any random neighbor cell for next step
        4. success =0 :Agent is able to find all mines on the board
            Terminate the game (Agent won the game!!)
        
        """ 
        self.count_total_moves = 0
        #Start game from random cell
        cell= self.start
        #self.print_move_on_board(tuple(cell),0)
        new_cell, new_clue = self.make_random_move(cell)   #Make a random move from first cell
        success, game_over ,self.mines_found =self.Improved_Inferences(new_cell,new_clue)
        cell= new_cell
        
        #Untill all mines are found
        #flag = self.mines_found != self.total_mines
        while self.mines_found != self.total_mines:
            #flag = self.mines_found != self.total_mines
            #print('The mines found are: ',self.mines_found)
            
            while success and self.mines_found != self.total_mines:
                self.count_total_moves +=1
                
            
                if success== 2:   # All neighboring cells are safe
                    
                    new_cell, new_clue = self.make_random_move(cell)
                    success, game_over ,self.mines_found =self.Improved_Inferences(new_cell,new_clue)
                    cell = new_cell
                    continue
                
                if success== 1:   # If agent is unable to decide then it should randomly pick the neighboring cell
                    new_cell, new_clue = self.make_random_move(cell)
                    success, game_over ,self.mines_found =self.Improved_Inferences(new_cell,new_clue)
                    cell=new_cell
                    continue
                    
                #If all neighboring cells are mine, choose the random cell from board
                if success == 3:
                    i = np.random.choice(self.dim)
                    j = np.random.choice(self.dim)
                    cell = (i,j)
                    new_cell, new_clue = self.make_random_move(cell)
                    success, game_over ,self.mines_found =self.Improved_Inferences(new_cell,new_clue)
                    cell = new_cell
                    continue
                    
                if self.mines_found == self.total_mines:   #agent successfully found all mines
                    success = 0
                    #flag = 0
                   # print('The Agent has successfully found all mines in steps:',self.count_total_moves)
                    exit()
        if success ==0:
            #flag =0
            exit()
             
    
                    
        return self.count_total_moves
              
    
    def end_game(self):
        """
        Agent has successfully identified all the mines of the game board.
        Lets end the game by collecting d,n, and mines found and use them for calculations

        """ 
        
        print('Hurray!!\n Agent successfully flagged out all the mines :)\n')
        print('for d:',d,'and for mine density:',n,'\n the total no of mines:',len(self.mines_found),'\n The Simple Agent is successfully able to find the mines in steps:',self.count_total_moves)

        self.output.add((d,n,len(self.mines_found),self.count_total_moves))
        self.choose_mine_cell.clear()   
        return d,n,len(self.mines_found),self.count_total_moves, self.output
    
    def calculations(self,d,n):
        """
        Once the agent has discovered total mines on the board, lets calculate:
        1. Final score = total no of mines discovered/total mines
        2. Safe cells score = total no  of safe cells identified/total safe cells

        """
        
        #Calculation of safely identified mines/total mines
        self.final_score =None
        self.final_score_1 =None
      
        #method-2
       # print('self.moves_made\n:',self.moves_made,'length:',len(self.moves_made))
       # print('self.mines_found\n:',self.mines_found,'length:',len(self.mines_found))
        self.t = self.mines_found.difference(self.moves_made)
        #print('the safely identified mines are:\n:',self.t, 'Its length is:',len(self.t))
        
        if len(self.total_mines):
            self.final_score_1 = len(self.t)/len(self.total_mines)
            print('The Final score_1 for d:',d,'\n and for mine density n:',n,'\n is:',len(self.t),'/',len(self.total_mines),'=',self.final_score_1)
       
             
        
        return self.final_score_1
          

    def start_game(self,d,n):
        """
        Representation of the mine sweeper board- create and print the board with given dimensions
        Select the start cell of the game randomly
        Check whether its mine or a safe cell
        Complete the search of all cells and calculate total no of mines

        """    
        self.board, self.total_mines, self.grid_array = self.obj2.board_representation(d,n)
        
        #Creating set containing all the index of the board dimension
        for i in range(self.dim):
            for j in range(self.dim):
                self.total_set.add((i,j))
        
        #Randomly creating first start cell
        i = np.random.choice(self.dim)
        j = np.random.choice(self.dim)
        self.start = (i,j)
        print('The start node is:',self.start)
        self.count_total_moves = self.add_knowledge(self.start)
        self.end_game()
        self.score1 = self.calculations(d,n)
        return len(self.moves_made),self.score1,self.count_total_moves
  
    def start_game_bonus(self,d,n):
        """
        Representation of the mine sweeper board- create and print the board with given dimensions
        Select the start cell of the game randomly
        Check whether its mine or a safe cell
        Complete the search of all cells and calculate total no of mines

        """    
        self.board, self.total_mines, self.grid_array = self.obj2.board_representation(d,n)
        
        #Creating set containing all the index of the board dimension
        for i in range(self.dim):
            for j in range(self.dim):
                self.total_set.add((i,j))
        
        #Randomly creating first start cell
        i = np.random.choice(self.dim)
        j = np.random.choice(self.dim)
        self.start = (i,j)
        print('The start node is:',self.start)
        self.count_total_moves = self.add_knowledge_bonus(self.start)
        self.end_game()
        self.score1 = self.calculations(d,n)
        return len(self.moves_made),self.score1,self.count_total_moves
    
        
        
        
        
     

In [10]:
#Start the minesweeper Game!!
d = input('Enter the size of dimension of the minesweeper d :')   
n=input('Enter the mine density n :')
obj5 = Improved_Agent(d,n)
obj5.start_game(d,n)

Enter the size of dimension of the minesweeper d :5
Enter the mine density n :0.1
The start node is: (0, 2)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 5 and for mine density: 0.1 
 the total no of mines: 4 
 The Simple Agent is successfully able to find the mines in steps: 12
The Final score_1 for d: 5 
 and for mine density n: 0.1 
 is: 2 / 4 = 0.5


(11, 0.5, 12)

In [11]:
#Start the minesweeper Game!!
d = input('Enter the size of dimension of the minesweeper d :')   
n=input('Enter the mine density n :')
obj5 = Improved_Agent(d,n)
obj5.start_game_bonus(d,n)

Enter the size of dimension of the minesweeper d :5
Enter the mine density n :0.1
The start node is: (1, 4)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 5 and for mine density: 0.1 
 the total no of mines: 4 
 The Simple Agent is successfully able to find the mines in steps: 10
The Final score_1 for d: 5 
 and for mine density n: 0.1 
 is: 2 / 4 = 0.5


(10, 0.5, 10)

In [ ]:
# Performance of Improved Agent
#Start the Game
record_steps = []
n_set = []
score_set_1 =[]
count_steps =[]
d = 20
#n = 0.1
for n in range(9):
    n = (n+1)/10
    
    n_set.append(n)
    obj1 = Improved_Agent(d,n)
    i,j,k=obj1.start_game(d,n)
    record_steps.append(i)
    score_set_1.append(j)
    count_steps.append(k)
    
print('n:', n_set)
print('steps:',record_steps)
print('score_1:',score_set_1)
print('count_steps:',count_steps)


In [ ]:
#Bonus Question: Knowing no of mines, Performance of Improved Agent
#Start the Game
record_steps = []
n_set = []
score_set_1 =[]
count_steps =[]
#score_set_2 =[]
d = 20
#n = 0.1
for n in range(9):
    n = (n+1)/10
    
    n_set.append(n)
    obj0 = Improved_Agent(d,n)
    i,j,k=obj0.start_game_bonus(d,n)
    record_steps.append(i)
    score_set_1.append(j)
    count_steps.append(k)
    
print('n:', n_set)
print('steps:',record_steps)
print('score_1:',score_set_1)
print('count_steps:',count_steps)
  

In [ ]:
# Performance of Improved Agent
#Start the Game
records = []
n_set = []
final_score =[]
count =[]

d = 10
#n = 0.1
for n in range(9):
    n = (n+1)/10
    n_set.append(n)
    score_set_1 =[]
    #count_steps =[]
    record_steps = []
    for r in range(50):
        obj1 = Improved_Agent(d,n)
       
        i,j,k=obj1.start_game(d,n)
        record_steps.append(i)
        score_set_1.append(j)
        #count_steps.append(k)
        
    records.append(mean(record_steps))
    final_score.append(mean(score_set_1))
    #count.append(mean(count_steps))
    
    
print('n:', n_set)
print('steps:',records)
print('score_1:',final_score)
#print('count_steps:',count)


The start node is: (1, 7)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 9 
 The Simple Agent is successfully able to find the mines in steps: 39
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 2 / 9 = 0.2222222222222222
The start node is: (2, 7)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 10 
 The Simple Agent is successfully able to find the mines in steps: 43
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 4 / 10 = 0.4
The start node is: (8, 6)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 12 
 The Simple Agent is successfully able to find the mines in steps: 36
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 7 / 12 = 0.5833333333333334
The start node is: (6, 7)
Hurray!!
 Agent successfully flagged out all the mines 

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 9 
 The Simple Agent is successfully able to find the mines in steps: 45
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 3 / 9 = 0.3333333333333333
The start node is: (2, 6)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 12 
 The Simple Agent is successfully able to find the mines in steps: 41
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 2 / 12 = 0.16666666666666666
The start node is: (6, 0)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.1 
 the total no of mines: 9 
 The Simple Agent is successfully able to find the mines in steps: 34
The Final score_1 for d: 10 
 and for mine density n: 0.1 
 is: 3 / 9 = 0.3333333333333333
The start node is: (4, 8)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 1

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 23 
 The Simple Agent is successfully able to find the mines in steps: 73
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 9 / 23 = 0.391304347826087
The start node is: (4, 4)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 22 
 The Simple Agent is successfully able to find the mines in steps: 66
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 4 / 22 = 0.18181818181818182
The start node is: (4, 8)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 22 
 The Simple Agent is successfully able to find the mines in steps: 59
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 12 / 22 = 0.5454545454545454
The start node is: (1, 2)
Hurray!!
 Agent successfully flagged out all the mines :)

for 

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 17 
 The Simple Agent is successfully able to find the mines in steps: 50
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 6 / 17 = 0.35294117647058826
The start node is: (1, 3)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 22 
 The Simple Agent is successfully able to find the mines in steps: 58
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 8 / 22 = 0.36363636363636365
The start node is: (9, 4)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.2 
 the total no of mines: 21 
 The Simple Agent is successfully able to find the mines in steps: 60
The Final score_1 for d: 10 
 and for mine density n: 0.2 
 is: 8 / 21 = 0.38095238095238093
The start node is: (5, 8)
Hurray!!
 Agent successfully flagged out all the mines :)

fo

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.3 
 the total no of mines: 29 
 The Simple Agent is successfully able to find the mines in steps: 63
The Final score_1 for d: 10 
 and for mine density n: 0.3 
 is: 12 / 29 = 0.41379310344827586
The start node is: (7, 2)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.3 
 the total no of mines: 35 
 The Simple Agent is successfully able to find the mines in steps: 83
The Final score_1 for d: 10 
 and for mine density n: 0.3 
 is: 11 / 35 = 0.3142857142857143
The start node is: (8, 1)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.3 
 the total no of mines: 31 
 The Simple Agent is successfully able to find the mines in steps: 77
The Final score_1 for d: 10 
 and for mine density n: 0.3 
 is: 12 / 31 = 0.3870967741935484
The start node is: (7, 3)
Hurray!!
 Agent successfully flagged out all the mines :)

f

...

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.6 
 the total no of mines: 63 
 The Simple Agent is successfully able to find the mines in steps: 110
The Final score_1 for d: 10 
 and for mine density n: 0.6 
 is: 17 / 63 = 0.2698412698412698
The start node is: (4, 7)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.6 
 the total no of mines: 58 
 The Simple Agent is successfully able to find the mines in steps: 86
The Final score_1 for d: 10 
 and for mine density n: 0.6 
 is: 22 / 58 = 0.3793103448275862
The start node is: (5, 1)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.6 
 the total no of mines: 59 
 The Simple Agent is successfully able to find the mines in steps: 97
The Final score_1 for d: 10 
 and for mine density n: 0.6 
 is: 19 / 59 = 0.3220338983050847
The start node is: (1, 1)
Hurray!!
 Agent successfully flagged out all the mines :)

f

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.6 
 the total no of mines: 60 
 The Simple Agent is successfully able to find the mines in steps: 108
The Final score_1 for d: 10 
 and for mine density n: 0.6 
 is: 24 / 60 = 0.4
The start node is: (9, 5)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.6 
 the total no of mines: 53 
 The Simple Agent is successfully able to find the mines in steps: 102
The Final score_1 for d: 10 
 and for mine density n: 0.6 
 is: 20 / 53 = 0.37735849056603776
The start node is: (4, 9)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.6 
 the total no of mines: 57 
 The Simple Agent is successfully able to find the mines in steps: 111
The Final score_1 for d: 10 
 and for mine density n: 0.6 
 is: 19 / 57 = 0.3333333333333333
The start node is: (8, 3)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.7 
 the total no of mines: 67 
 The Simple Agent is successfully able to find the mines in steps: 101
The Final score_1 for d: 10 
 and for mine density n: 0.7 
 is: 15 / 67 = 0.22388059701492538
The start node is: (9, 0)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.7 
 the total no of mines: 71 
 The Simple Agent is successfully able to find the mines in steps: 107
The Final score_1 for d: 10 
 and for mine density n: 0.7 
 is: 24 / 71 = 0.3380281690140845
The start node is: (1, 6)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.7 
 the total no of mines: 66 
 The Simple Agent is successfully able to find the mines in steps: 125
The Final score_1 for d: 10 
 and for mine density n: 0.7 
 is: 15 / 66 = 0.22727272727272727
The start node is: (5, 4)
Hurray!!
 Agent successfully flagged out all the mines :

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 87 
 The Simple Agent is successfully able to find the mines in steps: 113
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 29 / 87 = 0.3333333333333333
The start node is: (1, 3)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 78 
 The Simple Agent is successfully able to find the mines in steps: 108
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 22 / 78 = 0.28205128205128205
The start node is: (5, 7)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 81 
 The Simple Agent is successfully able to find the mines in steps: 111
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 19 / 81 = 0.2345679012345679
The start node is: (7, 9)
Hurray!!
 Agent successfully flagged out all the mines :)

Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 83 
 The Simple Agent is successfully able to find the mines in steps: 155
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 6 / 83 = 0.07228915662650602
The start node is: (1, 4)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 78 
 The Simple Agent is successfully able to find the mines in steps: 115
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 18 / 78 = 0.23076923076923078
The start node is: (8, 8)
Hurray!!
 Agent successfully flagged out all the mines :)

for d: 10 and for mine density: 0.8 
 the total no of mines: 74 
 The Simple Agent is successfully able to find the mines in steps: 118
The Final score_1 for d: 10 
 and for mine density n: 0.8 
 is: 12 / 74 = 0.16216216216216217
The start node is: (2, 9)
Hurray!!
 Agent successfully flagged out all the mines :

In [ ]:
# Performance of Improved Agent(Bonus)
#Start the Game
records = []
n_set = []
final_score =[]
count =[]

d = 10
#n = 0.1
for n in range(9):
    n = (n+1)/10
    n_set.append(n)
    score_set_1 =[]
    #count_steps =[]
    record_steps = []
    for r in range(50):
        obj1 = Improved_Agent(d,n)
       
        i,j,k=obj1.start_game_bonus(d,n)
        record_steps.append(i)
        score_set_1.append(j)
        #count_steps.append(k)
        
    records.append(mean(record_steps))
    final_score.append(mean(score_set_1))
    #count.append(mean(count_steps))
    
    
print('n:', n_set)
print('steps:',records)
print('score_1:',final_score)
#print('count_steps:',count)
